The following code is based on the work: A Deeper Analysis of Adversarial Examples in Intrusion Detection by Mohamed Amine Merzouk, Frederic Cuppens,Nora Boulahia-Cuppens3, and Reda Yaich. Official github [repo](https://github.com/mamerzouk/adversarial_analysis/tree/master).

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import copy
import time as time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2

from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent, DeepFool, CarliniL2Method, CarliniLInfMethod
from art.estimators.classification import PyTorchClassifier

%matplotlib inline

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

# NSL-KDD dataset

In [122]:
#Downloading and extracting the dataset if it doesn't exist
!if [ ! -d "./NSL-KDD" ]; then wget http://205.174.165.80/CICDataset/NSL-KDD/Dataset/NSL-KDD.zip; mkdir NSL-KDD; unzip NSL-KDD.zip -d NSL-KDD; fi

In [123]:
#Importing the training and testing datasets from .CSV to Pandas DataFrames
features = ['1 Duration', '2 Protocol-type : ', '3 Service : ', '4 Flag : ', '5 Src-bytes', '6 Dst-bytes', '7 Land', '8 Wrong-fragment', '9 Urgent', '10 Hot', '11 Num-failed-logins', '12 Logged-in', '13 Num-compromised', '14 Root-shell', '15 Su-attempted', '16 Num-root', '17 Num-file-creations', '18 Num-shells', '19 Num-access-files', '20 Num-outbound-cmds', '21 Is-host-login', '22 Is-guest-login', '23 Count', '24 Srv-count', '25 Serror-rate', '26 Srv-serror-rate', '27 Rerror-rate', '28 Srv-rerror-rate', '29 Same-srv-rate', '30 Diff-srv-rate', '31 Srv-diff-host-rate', '32 Dst-host-count', '33 Dst-host-srv-count', '34 Dst-host-same-srv-rate', '35 Dst-host-diff-srv-rate', '36 Dst-host-same-src-port-rate', '37 Dst-host-srv-diff-host-rate', '38 Dst-host-serror-rate', '39 Dst-host-srv-serror-rate', '40 Dst-host-rerror-rate', '41 Dst-host-srv-rerror-rate', '42 Attack_type', '43 Difficulty']
df_training = pd.read_csv('./NSL-KDD/KDDTrain+.txt', names=features)
df_testing = pd.read_csv('./NSL-KDD/KDDTest+.txt', names=features)
# Stack the training and test sets
data = pd.concat([df_training, df_testing], axis=0)

In [124]:
data

,1 Duration,2 Protocol-type :,3 Service :,4 Flag :,5 Src-bytes,6 Dst-bytes,7 Land,8 Wrong-fragment,9 Urgent,10 Hot,11 Num-failed-logins,12 Logged-in,13 Num-compromised,14 Root-shell,15 Su-attempted,16 Num-root,17 Num-file-creations,18 Num-shells,19 Num-access-files,20 Num-outbound-cmds,21 Is-host-login,22 Is-guest-login,23 Count,24 Srv-count,25 Serror-rate,26 Srv-serror-rate,27 Rerror-rate,28 Srv-rerror-rate,29 Same-srv-rate,30 Diff-srv-rate,31 Srv-diff-host-rate,32 Dst-host-count,33 Dst-host-srv-count,34 Dst-host-same-srv-rate,35 Dst-host-diff-srv-rate,36 Dst-host-same-src-port-rate,37 Dst-host-srv-diff-host-rate,38 Dst-host-serror-rate,39 Dst-host-srv-serror-rate,40 Dst-host-rerror-rate,41 Dst-host-srv-rerror-rate,42 Attack_type,43 Difficulty
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal,21
22540,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal,21
22541,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,back,15
22542,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [125]:
# Drop the last column (which might be the difficulty, so it's useless)
data.drop('43 Difficulty', inplace=True, axis=1)
# Drop the 19th column wich is full of 0, so has std=0. which causes issues for the normalization
data.drop('20 Num-outbound-cmds', inplace=True, axis=1)

In [126]:
# Transform the nominal attribute "Attack type" into binary (0 : normal / 1 : attack)
labels = (data['42 Attack_type'] != 'normal').astype('int64')
data['42 Labels'] = labels
data.drop('42 Attack_type', inplace=True, axis=1)

In [127]:
# One Hot Encode the 3 first nominal attributes and drop them
for i in ['4 Flag : ', '3 Service : ', '2 Protocol-type : ']:
    # Create the One Hot Encode DataFrame
    dum = pd.get_dummies(data[i])
    # Insert into the dataset DataFrame by Series
    for column_name in list(dum.columns):
        data.insert(1, str(i)+column_name, dum[column_name])
        data[str(i)+column_name] = data[str(i)+column_name].astype('int64')
    # Drop the old attribute's column
    data.drop(i, inplace=True, axis=1)

In [128]:
# Split training and test sets
df_training = data[:df_training.shape[0]]    
df_testing = data[df_training.shape[0]:]

In [129]:
# Min-Max normalization on the non binary features
for i in ['1 Duration', '5 Src-bytes', '6 Dst-bytes', '8 Wrong-fragment', '9 Urgent', '10 Hot', '11 Num-failed-logins', '13 Num-compromised', '15 Su-attempted', '16 Num-root', '17 Num-file-creations', '18 Num-shells', '19 Num-access-files', '23 Count', '24 Srv-count', '25 Serror-rate', '26 Srv-serror-rate', '27 Rerror-rate', '28 Srv-rerror-rate', '29 Same-srv-rate', '30 Diff-srv-rate', '31 Srv-diff-host-rate', '32 Dst-host-count', '33 Dst-host-srv-count', '34 Dst-host-same-srv-rate', '35 Dst-host-diff-srv-rate', '36 Dst-host-same-src-port-rate', '37 Dst-host-srv-diff-host-rate', '38 Dst-host-serror-rate', '39 Dst-host-srv-serror-rate', '40 Dst-host-rerror-rate', '41 Dst-host-srv-rerror-rate']:
    # The min and max are only computed from the training set
    min = df_training[i].min()
    max = df_training[i].max()
    df_training[i] = ((df_training[i] - min) / (max - min)) 
    df_testing[i] = ((df_testing[i] - min) / (max - min)) 
     

/tmp/ipykernel_120859/3046827848.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training[i] = ((df_training[i] - min) / (max - min))
/tmp/ipykernel_120859/3046827848.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_testing[i] = ((df_testing[i] - min) / (max - min))
/tmp/ipykernel_120859/3046827848.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [130]:
# Get NumPy arrays from DataFrames
nd_training = df_training.values
nd_testing = df_testing.values

In [131]:
nd_training.shape, nd_testing.shape

((125973, 122), (22544, 122))

In [132]:
nd_training

array([[0.  , 0.  , 1.  , ..., 0.05, 0.  , 0.  ],
       [0.  , 1.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 1.  ],
       ...,
       [0.  , 0.  , 1.  , ..., 0.01, 0.  , 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 1.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ]])

In [133]:
# Separating arguments (x) from labelss (y)
x_train = nd_training[:, :-1]
y_train = nd_training[:, -1]
x_test = nd_testing[:, :-1]
y_test = nd_testing[:, -1]

# Make a copy of the data set as NumPy arrays
x_train_np = x_train.copy()
y_train_np = y_train.copy()
x_test_np = x_test.copy()
y_test_np = y_test.copy()

In [134]:
# Convert from numpy array to torch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

In [135]:
class Network(nn.Module):
    ''' A basic neural network model '''
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()         #python2 : super(MLP, self).__init__()
        #defining the network's operations
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], output_size)

    def forward(self, x, softmax=False): 
        a = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x.float())))))
        if softmax:
            y_pred = F.softmax(a, dim=1)
        else:
            y_pred = a

        return y_pred

In [136]:
def evaluate_predictions(predictions, real):
    ''' Evaluates the accuracy of the predictions'''
    n_correct = torch.eq(predictions, real).sum().item()
    accuracy = n_correct / len(predictions) * 100
    return accuracy

In [138]:
# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on : {}".format(device))

Training on : cuda


In [139]:
# Initialising the model
input_size=x_train.shape[1]
print('input size is ', input_size)
hidden_size=[256,256]
output_size=2
model = Network(input_size, hidden_size, output_size)

input size is  121


In [140]:
# Transfering model and data to GPU
model = model.to(device)
x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

In [141]:
model

Network(
  (fc1): Linear(in_features=121, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)

In [142]:
# Setting the Loss function and Adam learning rate
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)

In [146]:
model.load_state_dict(torch.load("./model.pytorch"))

<All keys matched successfully>

# Adversarial attack

In [147]:
adv_feat_stats = pd.DataFrame(index=df_training.columns[:-1])

adv_results = pd.DataFrame(index=['Accuracy', 
                                  'Mean perturbed features   [Mean L0]', 
                                  'Max perturbed features    [Max  L0]', 
                                  'Mean Euclidiant distance  [Mean L2]', 
                                  'Max Euclidiant distance   [Max  L2]', 
                                  'Mean Maximum perturbation [Mean Li]', 
                                  'Max Maximum perturbation  [Max  Li]'])

adv_inv = pd.DataFrame(index=['Invalid value range',
                              'Invalid binary values',
                              'Invalid class belonging'])

In [148]:
def adv_norms(x_test_cpu, adversarial_examples_cpu):
    mean_l0 = np.mean(np.sum(x_test_cpu != adversarial_examples_cpu, axis=1))
    max_l0 = np.max(np.sum(x_test_cpu != adversarial_examples_cpu, axis=1))
    mean_l2 = np.mean(np.sum(np.power(x_test_cpu - adversarial_examples_cpu, 2), axis=1, keepdims=True))
    max_l2 = np.max(np.sum(np.power(x_test_cpu - adversarial_examples_cpu, 2), axis=1, keepdims=True))
    mean_li = np.mean(np.max(np.abs(x_test_cpu - adversarial_examples_cpu), axis=1, keepdims=True))
    max_li = np.max(np.max(np.abs(x_test_cpu - adversarial_examples_cpu), axis=1, keepdims=True))
    return [mean_l0, max_l0, mean_l2, max_l2, mean_li, max_li]

In [149]:
def adv_criteria(x_test_cpu, adversarial_examples_cpu):
    # Verify value ranges
    min = x_test_cpu.min(axis=1, keepdims=True)
    max = x_test_cpu.max(axis=1, keepdims=True)
    adv_range = (adversarial_examples_cpu < min) | (adversarial_examples_cpu > max)
    adv_range = adv_range.any(axis=1, keepdims=True)
    adv_range = adv_range.sum(axis=0)
    #print("proportion of out-of-range values : {:.2f}% | {}/{}".format(adv[0]*100/x_test.shape[0], adv[0], x_test.shape[0]))

    # Binary values
    binary_feat_ind = list(range(1,85)) + [87, 92, 94, 100, 101]
    adv_bin = adversarial_examples_cpu[:, binary_feat_ind]
    adv_bin = (adv_bin != 1) & ( adv_bin != 0)
    adv_bin = adv_bin.any(axis=1, keepdims=True)
    adv_bin = adv_bin.sum(axis=0)
    #print("proportion of non-binary values : {:.2f}% | {}/{}".format(adv[0]*100/x_test.shape[0], adv[0], x_test.shape[0]))

    # Multi class
    adv1 = adversarial_examples_cpu[:, 1:4] != 0
    adv1 = adv1.astype(int).sum(axis=1, keepdims=True) != 1
    adv1 = adv1.sum(axis=1, keepdims=True)

    adv2 = adversarial_examples_cpu[:, 4:74] != 0
    adv2 = adv2.astype(int).sum(axis=1, keepdims=True) != 1
    adv2 = adv2.sum(axis=1, keepdims=True)

    adv3 = adversarial_examples_cpu[:, 74:85] != 0
    adv3 = adv3.astype(int).sum(axis=1, keepdims=True) != 1
    adv3 = adv3.sum(axis=1, keepdims=True)

    adv_cat = adv1 | adv2 | adv3
    adv_cat = adv_cat.sum(axis=0)
    #print("proportion of multiple category values : {:.2f}% | {}/{}".format(adv[0]*100/x_test.shape[0], adv[0], x_test.shape[0]))

    return [adv_range[0]*100/x_test.shape[0], adv_bin[0]*100/x_test.shape[0], adv_cat[0]*100/x_test.shape[0]]

In [183]:
positive_examples = df_testing[df_testing['42 Labels'] == 1].values
x_test = torch.from_numpy((positive_examples[:, :-1])).float()
y_test = torch.from_numpy((positive_examples[:, -1])).float()
x_test = x_test.to(device)
y_test = y_test.to(device)

In [184]:
# Applying the PyTorch wrapper
classifier = PyTorchClassifier(model=model, loss=criterion, optimizer=optimizer, input_shape=input_size, nb_classes=output_size)

In [185]:
# The model performance on untouched attack samples.
_, predictions_clean = model(x_test, softmax=True).max(dim=1)
accuracy_clean = evaluate_predictions(predictions=predictions_clean.long(), real=y_test)
print(accuracy_clean)
attack='Clean'

# Exporting the clean examples in a .xlsx file
#excel_writer = pd.ExcelWriter("adversarial_examples.xlsx", engine='openpyxl') 
#xlsx_export = pd.DataFrame(np.hstack((x_test.cpu().numpy(),y_test.cpu().numpy().reshape(y_test.shape[0], 1))), columns=data.columns)
#xlsx_export.to_excel(excel_writer, sheet_name=attack)

# Exporting the clean examples in a .csv file
pd.DataFrame(np.hstack((x_test.cpu().numpy(),y_test.cpu().numpy().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("clean_examples.csv")

x_test_cpu = np.array(x_test.cpu())
adv_results[attack] = [accuracy_clean] + adv_norms(x_test_cpu, x_test_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, x_test_cpu)

print(adv_results[attack])
print(adv_inv[attack])

80.17610847034989
Accuracy                               80.176108
Mean perturbed features   [Mean L0]     0.000000
Max perturbed features    [Max  L0]     0.000000
Mean Euclidiant distance  [Mean L2]     0.000000
Max Euclidiant distance   [Max  L2]     0.000000
Mean Maximum perturbation [Mean Li]     0.000000
Max Maximum perturbation  [Max  Li]     0.000000
Name: Clean, dtype: float64
Invalid value range        0.0
Invalid binary values      0.0
Invalid class belonging    0.0
Name: Clean, dtype: float64


# Fast Gradient Sign Method

In [186]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.1,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )

adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'FGSM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)
# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM1.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.8087%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.52     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.52     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.808696
Mean perturbed features   [Mean L0]    103.030312
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      1.030303
Max Euclidiant distance   [Max  L2]      1.209999
Mean Maximum perturbation [Mean Li]      0.085187
Max Maximum perturbation  [Max  Li]      0.100000
Name: FGSM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: FGSM, dtype: float64


In [187]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.2,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )

adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'FGSM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)
# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM2.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.9568%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.52     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.52     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.956752
Mean perturbed features   [Mean L0]    103.030312
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      4.121210
Max Euclidiant distance   [Max  L2]      4.839997
Mean Maximum perturbation [Mean Li]      0.170373
Max Maximum perturbation  [Max  Li]      0.200000
Name: FGSM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: FGSM, dtype: float64


In [188]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.3,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )

adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'FGSM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)
# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM3.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.9568%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.52     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.52     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.956752
Mean perturbed features   [Mean L0]    103.030312
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      9.272736
Max Euclidiant distance   [Max  L2]     10.890010
Mean Maximum perturbation [Mean Li]      0.255560
Max Maximum perturbation  [Max  Li]      0.300000
Name: FGSM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: FGSM, dtype: float64


In [189]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.05,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )

adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'FGSM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)
# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM4.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 35.1827%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.52     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.18      0.26     12833
weighted avg       1.00      0.35      0.52     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                35.182732
Mean perturbed features   [Mean L0]    103.030312
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      0.257576
Max Euclidiant distance   [Max  L2]      0.302500
Mean Maximum perturbation [Mean Li]      0.042593
Max Maximum perturbation  [Max  Li]      0.050000
Name: FGSM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: FGSM, dtype: float64


# BIM

In [190]:
#BIM
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.05, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'BIM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)


# Exporting the adversarial examples in a .xlsx file
#xlsx_export = pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns)
#xlsx_export['Adversarial prediction'] = predictions_adv.cpu().numpy().reshape(y_test.shape[0], 1)
#xlsx_export.to_excel(excel_writer, sheet_name=attack)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM1.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    103.043871
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      0.242522
Max Euclidiant distance   [Max  L2]      0.302500
Mean Maximum perturbation [Mean Li]      0.042593
Max Maximum perturbation  [Max  Li]      0.050000
Name: BIM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: BIM, dtype: float64


In [191]:
#BIM
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.1, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'BIM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)


# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM2.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    103.046988
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      0.861515
Max Euclidiant distance   [Max  L2]      1.171971
Mean Maximum perturbation [Mean Li]      0.085187
Max Maximum perturbation  [Max  Li]      0.100000
Name: BIM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: BIM, dtype: float64


In [192]:
#BIM
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.2, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'BIM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM3.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    103.046988
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      0.861516
Max Euclidiant distance   [Max  L2]      1.171974
Mean Maximum perturbation [Mean Li]      0.085189
Max Maximum perturbation  [Max  Li]      0.100005
Name: BIM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: BIM, dtype: float64


In [193]:
#BIM
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.3, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'BIM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)


# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM4.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    103.046988
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      0.861516
Max Euclidiant distance   [Max  L2]      1.171974
Mean Maximum perturbation [Mean Li]      0.085189
Max Maximum perturbation  [Max  Li]      0.100005
Name: BIM, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: BIM, dtype: float64


# PGD

In [194]:
#PGD 
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.05,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'PGD'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)


# Exporting the adversarial examples in a .xlsx file
#xlsx_export = pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns)
#xlsx_export['Adversarial prediction'] = predictions_adv.cpu().numpy().reshape(y_test.shape[0], 1)
#xlsx_export.to_excel(excel_writer, sheet_name=attack)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD1.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    102.656511
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      0.253195
Max Euclidiant distance   [Max  L2]      0.302500
Mean Maximum perturbation [Mean Li]      0.042593
Max Maximum perturbation  [Max  Li]      0.050000
Name: PGD, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: PGD, dtype: float64


In [195]:
#PGD 
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.1,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'PGD'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)


# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD2.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    102.910076
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      1.015563
Max Euclidiant distance   [Max  L2]      1.209999
Mean Maximum perturbation [Mean Li]      0.085187
Max Maximum perturbation  [Max  Li]      0.100000
Name: PGD, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: PGD, dtype: float64


In [196]:
#PGD 
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.2,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'PGD'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)



# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD3.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    102.959713
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      4.042997
Max Euclidiant distance   [Max  L2]      4.839997
Mean Maximum perturbation [Mean Li]      0.170373
Max Maximum perturbation  [Max  Li]      0.200000
Name: PGD, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: PGD, dtype: float64


In [197]:
#PGD 
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.3,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)
attack = 'PGD'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)


# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD4.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

PGD - Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 34.6373%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.35      0.51     12833

    accuracy                           0.35     12833
   macro avg       0.50      0.17      0.26     12833
weighted avg       1.00      0.35      0.51     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                                34.637263
Mean perturbed features   [Mean L0]    103.025092
Max perturbed features    [Max  L0]    121.000000
Mean Euclidiant distance  [Mean L2]      9.057258
Max Euclidiant distance   [Max  L2]     10.890010
Mean Maximum perturbation [Mean Li]      0.255560
Max Maximum perturbation  [Max  Li]      0.300000
Name: PGD, dtype: float64
Invalid value range        85.186628
Invalid binary values      85.186628
Invalid class belonging    85.186628
Name: PGD, dtype: float64


# CW L_2

In [201]:
# Creating the adversarial examples crafter
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)

# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

# The transformation to tanh space introduce some small perturbation, we remove it to get the exact statistics
adversarial_examples = pd.DataFrame(adversarial_examples)
adversarial_examples[(np.abs(adversarial_examples - x_test_cpu) < 10e-6)] = x_test_cpu
adversarial_examples = adversarial_examples.values

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'CW2'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_CW21.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

C&W L_2:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 18.1485%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.18      0.31     12833

    accuracy                           0.18     12833
   macro avg       0.50      0.09      0.15     12833
weighted avg       1.00      0.18      0.31     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                               18.148523
Mean perturbed features   [Mean L0]    14.014026
Max perturbed features    [Max  L0]    22.000000
Mean Euclidiant distance  [Mean L2]     1.668435
Max Euclidiant distance   [Max  L2]     9.485065
Mean Maximum perturbation [Mean Li]     0.577300
Max Maximum perturbation  [Max  Li]     1.494115
Name: CW2, dtype: float64
Invalid value range        96.571340
Invalid binary values      99.992208
Invalid class belonging     0.000000
Name: CW2, dtype: float64


In [205]:
# Creating the adversarial examples crafter
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.1,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)

# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

# The transformation to tanh space introduce some small perturbation, we remove it to get the exact statistics
adversarial_examples = pd.DataFrame(adversarial_examples)
adversarial_examples[(np.abs(adversarial_examples - x_test_cpu) < 10e-6)] = x_test_cpu
adversarial_examples = adversarial_examples.values

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'CW2'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW22.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

C&W L_2:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 18.0784%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.18      0.31     12833

    accuracy                           0.18     12833
   macro avg       0.50      0.09      0.15     12833
weighted avg       1.00      0.18      0.31     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                               18.078392
Mean perturbed features   [Mean L0]    14.001013
Max perturbed features    [Max  L0]    22.000000
Mean Euclidiant distance  [Mean L2]     1.695928
Max Euclidiant distance   [Max  L2]    10.695344
Mean Maximum perturbation [Mean Li]     0.583645
Max Maximum perturbation  [Max  Li]     1.829102
Name: CW2, dtype: float64
Invalid value range         96.134965
Invalid binary values      100.000000
Invalid class belonging      0.000000
Name: CW2, dtype: float64


In [206]:
# Creating the adversarial examples crafter
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=20,
                                      max_iter=20,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)

# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

# The transformation to tanh space introduce some small perturbation, we remove it to get the exact statistics
adversarial_examples = pd.DataFrame(adversarial_examples)
adversarial_examples[(np.abs(adversarial_examples - x_test_cpu) < 10e-6)] = x_test_cpu
adversarial_examples = adversarial_examples.values

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'CW2'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW23.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

C&W L_2:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 17.5173%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.18      0.30     12833

    accuracy                           0.18     12833
   macro avg       0.50      0.09      0.15     12833
weighted avg       1.00      0.18      0.30     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                               17.517338
Mean perturbed features   [Mean L0]    14.045741
Max perturbed features    [Max  L0]    22.000000
Mean Euclidiant distance  [Mean L2]     1.618345
Max Euclidiant distance   [Max  L2]     9.485065
Mean Maximum perturbation [Mean Li]     0.567012
Max Maximum perturbation  [Max  Li]     1.494115
Name: CW2, dtype: float64
Invalid value range        96.516793
Invalid binary values      99.992208
Invalid class belonging     0.000000
Name: CW2, dtype: float64


In [207]:
# Creating the adversarial examples crafter
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.1,
                                      binary_search_steps=10,
                                      max_iter=100,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)

# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

# The transformation to tanh space introduce some small perturbation, we remove it to get the exact statistics
adversarial_examples = pd.DataFrame(adversarial_examples)
adversarial_examples[(np.abs(adversarial_examples - x_test_cpu) < 10e-6)] = x_test_cpu
adversarial_examples = adversarial_examples.values

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate_predictions(predictions=predictions_adv.long(), real=y_test)

attack = 'CW2'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW24.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

C&W L_2:   0%|          | 0/101 [00:00<?, ?it/s]

Final Training Accuracy: 80.1761%
Final Testing Accuracy : 15.6783%
Classification Report :
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.16      0.27     12833

    accuracy                           0.16     12833
   macro avg       0.50      0.08      0.14     12833
weighted avg       1.00      0.16      0.27     12833



/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` paramete

Accuracy                               15.678329
Mean perturbed features   [Mean L0]    14.098496
Max perturbed features    [Max  L0]    22.000000
Mean Euclidiant distance  [Mean L2]     1.618843
Max Euclidiant distance   [Max  L2]     6.939663
Mean Maximum perturbation [Mean Li]     0.563087
Max Maximum perturbation  [Max  Li]     1.359501
Name: CW2, dtype: float64
Invalid value range        96.212889
Invalid binary values      99.968830
Invalid class belonging     0.000000
Name: CW2, dtype: float64
